In [10]:
load("skeinslib.sage")

In [11]:
def is_highest(x, v, ordering):
    for vect in x:
        if ordering[vect] > ordering[v]:
            return False
    return True

def get_highest(x, ordering):
    for y in x:
        if is_highest(x, y, ordering):
            return y

def get_symbols(gamma, shell_level, order_func):
    '''
    OLD DESCRIPTION
    Returns a dict of linear relations between lattice points for a specified
    shell level.

    Each ordered pair of lattice points determines a relation between four other
    lattice points, where lattice points correspond to generators of the empty
    part of the skein module.

    Requires integer shell_level, and function order_func : int -> (dict, list)
    which should produce: a dictionary with keys being lattice points in a
    certain shell level, and values being their position in some sequential
    ordering specified by the function (this is required to map lattice points
    to indices of vectors in the space they span); and a list of lattice points
    in this order (required to produce the four related lattice points using
    basic linalg).

    Performs a double loop through the lattice, obtains the relation between
    four points for each pair of points, and discards trivial or out-of-range
    relations.
    '''

    symbols = {}
    N = (2*shell_level + 1)*(shell_level + 1) - shell_level # Total lattice pts.
    ordering = order_func(shell_level) # Dict and list of order

    #Unpack the matrix gamma.
    a = gamma[0, 0]
    b = gamma[0, 1]
    c = gamma[1, 0]
    d = gamma[1, 1]

    q = var('q') # Must be defined here to alllow compiled sage.

    for p_0 in ordering.keys():
        for p_1 in ordering.keys():
            #Unpack the points
            r = p_0[0]
            s = p_0[1]
            t = p_1[0]
            u = p_1[1]

            #A constant appearing in our coefficients, we compute it in advance
            C = (-t*(t-1)*a*c - u*(u-1)*b*d)/2 - t*u*c*b

            # The linear relation is between the four lattice points below:
            x_0 = force_into_fundamental_domain(vector(ZZ, p_0 + p_1, immutable=True))
            x_1 = force_into_fundamental_domain(vector(ZZ, p_0 - p_1, immutable=True))
            x_2 = force_into_fundamental_domain(vector(ZZ, p_0 + p_1*gamma.T, immutable=True))
            x_3 = force_into_fundamental_domain(vector(ZZ, p_0 - p_1*gamma.T, immutable=True))

            # Check the relations are not out of range.
            if x_0 in ordering.keys() and x_1 in ordering.keys() and x_2 in ordering.keys() and x_3 in ordering.keys():

                # Compute the coefficients in the relation.
                Q_0 = q**(-s*t)
                Q_1 = q**(s*t)
                Q_2 = -q**(C - r*(c*t + d*u))
                Q_3 = -q**(C + r*(c*t + d*u))
                
                coeffs = [Q_0, Q_1, Q_2, Q_3]
                
                x = [x_0, x_1, x_2, x_3]
                
                relation = list(zip(coeffs, x))
                
                highest_coeffs = [r[0] if is_highest(x, r[1], ordering) else 0 for r in relation]
                
                coeff = sum(highest_coeffs)
                
                x_highest = get_highest(x, ordering)
                
                #rel = vector(K, [coeff if i == ordering[x_highest] else 0 for i in range(N)], sparse=True)

                # Check the relation is not trivial, then append.
                if not coeff.is_zero():
                    symbols[(p_0, p_1)] = x_highest

    return symbols

def compute_from_symbols(gamma, shell_levels, order_func, interactive):
    dimensions = []
    for shell_level in range(shell_levels):
        ordering = order_func(shell_level)
        symbols = get_symbols(gamma, shell_level, order_func)
        #print(symbols)
        symbols = set(symbols.values())
        lattice = set(order_func(shell_level).keys())
        #print(lattice.difference(symbols))
        spanning_set = [ordering[l] for l in lattice.difference(symbols)]
        #print(symbols)
        #print(spanning_set)
        if interactive:
            print("Dimension estimate (using symbols) for empty skein part at level %d: %d" % (shell_level, len(spanning_set)))
            print_generators(shell_level, spanning_set, order_func)
        dimensions.append(len(spanning_set))
    return dimensions
        

In [6]:
S = matrix(ZZ, 2, [0, -1, 1, 0])
T = matrix(ZZ, 2, [1, 1, 0, 1])
I = matrix(ZZ, 2, [1, 0, 0, 1])

In [71]:
compute_from_symbols(T, 7, order_by_shell_level, True)

Dimension estimate (using symbols) for empty skein part at level 0: 1
x 

Dimension estimate (using symbols) for empty skein part at level 1: 3
x | . 
- x x 
. | . 

Dimension estimate (using symbols) for empty skein part at level 2: 4
x . | . . 
. x | . . 
- - x x - 
. . | . . 
. . | . . 

Dimension estimate (using symbols) for empty skein part at level 3: 5
x . . | . . . 
. x . | . . . 
. . x | . . . 
- - - x x - - 
. . . | . . . 
. . . | . . . 
. . . | . . . 

Dimension estimate (using symbols) for empty skein part at level 4: 6
x . . . | . . . . 
. x . . | . . . . 
. . x . | . . . . 
. . . x | . . . . 
- - - - x x - - - 
. . . . | . . . . 
. . . . | . . . . 
. . . . | . . . . 
. . . . | . . . . 

Dimension estimate (using symbols) for empty skein part at level 5: 7
x . . . . | . . . . . 
. x . . . | . . . . . 
. . x . . | . . . . . 
. . . x . | . . . . . 
. . . . x | . . . . . 
- - - - - x x - - - - 
. . . . . | . . . . . 
. . . . . | . . . . . 
. . . . . | . . . . . 
. . . . . | .

[1, 3, 4, 5, 6, 7, 8]

In [56]:
compute_reduced_matrix(S, 3, True)

Calculating relations for level 3 (25 lattice points) ...
Found 76 (non-independent) relations. Reducing ...
Calculating relations for level 2 (13 lattice points) ...
Found 24 (non-independent) relations. Reducing ...
Calculating relations for level 1 (5 lattice points) ...
Found 4 (non-independent) relations.

Dimension estimate for empty skein part at level 1: 3.

Visualisation:

x x . 
- x - 
. | . 

Dimension estimate for empty skein part at level 2: 4.

Visualisation:

. x | . . 
. x x . . 
- - x - - 
. . | . . 
. . | . . 

Dimension estimate for empty skein part at level 3: 4.

Visualisation:

. . . | . . . 
. . x | . . . 
. . x x . . . 
- - - x - - - 
. . . | . . . 
. . . | . . . 
. . . | . . . 



(104 x 25 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field,
 104 x 25 sparse matrix over Fraction Field of Sparse Univariate Polynomial Ring in q over Rational Field,
 [3, 4, 4])

In [26]:
type(list(get_symbols(S, 2, order_by_shell_level).values()))

<class 'list'>

In [17]:
load("skeinslib.sage")

In [14]:
compute_reduced_matrix(T, 4, True)

Calculating relations for level 4 (41 lattice points) ...
Found 182 (non-independent) relations. Reducing ...
Calculating relations for level 3 (25 lattice points) ...
Found 77 (non-independent) relations. Reducing ...
Calculating relations for level 2 (13 lattice points) ...
Found 23 (non-independent) relations. Reducing ...
Calculating relations for level 1 (5 lattice points) ...
Found 3 (non-independent) relations.



TypeError: no conversion of this rational to integer

In [18]:
M = Matrix(K, [[sqrt(q),q],[1,0]])

TypeError: cannot convert sqrt(q)/1.00000000000000 to an element of Fraction Field of Sparse Univariate Polynomial Ring in q over Complex Field with 53 bits of precision